In [1]:
import findspark
findspark.init()

In [17]:
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.master("local").appName("Test Spark").config("spark.some.config.option", "some-value").getOrCreate()

In [18]:
sc = spark.sparkContext

In [19]:
spark

In [20]:
sqlcontext = SQLContext(sc)

In [21]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [22]:
data = sqlcontext.read.csv(path='Agriculture-Department.csv', header = True, inferSchema = True)

In [8]:
#

NameError: name 'spark' is not defined

In [23]:
data.show()

+--------------------+-----------+----------+------+-----------+--------------+----------------+---+--------------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------+-------+
|  Application Status|Merit Score|  Domicile|Gender|   Province|Admission Test|Admission Test 2|SSC|TerminalDegree|Optional Subject 1|Optional Subject 2|Optional Subject 3|Optional Subject 4|Optional Subject 5|Document Verified|English Marks|Offered|
+--------------------+-----------+----------+------+-----------+--------------+----------------+---+--------------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------+-------+
|Application Received|    908.813|      OPEN|  male|Baluchistan|             0|               0|594|           702|                 0|                 0|                 0|                 0|                 0|                0|            0|     

In [28]:
data2 =data.select(data.TerminalDegree,data.SSC.alias('label'))
train, test = data2.randomSplit([0.9,0.1])

In [29]:
assembler=VectorAssembler().setInputCols(['TerminalDegree',]).setOutputCol('features')
train01 = assembler.transform(train)
train02 = train01.select("features","label")
train02.show(5)

+--------+-----+
|features|label|
+--------+-----+
|   [0.0]|    0|
| [483.0]|  652|
| [487.0]|  759|
| [495.0]|  615|
| [497.0]|  578|
+--------+-----+
only showing top 5 rows



In [30]:
lr = LinearRegression()
model = lr.fit(train02)

In [31]:
test01 = assembler.transform(test)
test02 = test01.select('features', 'label')
test03 = model.transform(test02)
test03.show(5)

+--------+-----+-----------------+
|features|label|       prediction|
+--------+-----+-----------------+
| [481.0]|  508|611.2325972791881|
| [483.0]|  657|612.4880594350288|
| [487.0]|  686|614.9989837467101|
| [515.0]|  550|632.5754539284795|
| [520.0]|  581|635.7141093180812|
+--------+-----+-----------------+
only showing top 5 rows



In [37]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import polyfit

In [38]:
import chart_studio.plotly as py
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode='markers',
        name='Original_Test',
    ))
fig.add_trace(
    go.Scatter(
        x=x,
        y=y_pred,
        name='Predicted'
    ))
fig.update_layout(
    title="Linear Regression",
    xaxis_title="Displacement",
    yaxis_title="Horse Power",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

NameError: name 'x' is not defined

In [39]:
evaluator = RegressionEvaluator()
print(evaluator.evaluate(test03,
{evaluator.metricName: "r2"})
)
print(evaluator.evaluate(test03,
{evaluator.metricName: "mse"})
)
print(evaluator.evaluate(test03,
{evaluator.metricName: "rmse"})
)
print(evaluator.evaluate(test03,
{evaluator.metricName: "mae"})
)

0.4544376523031217
6090.572336468868
78.04211899012525
60.64943273288574


In [ ]:
#testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())

In [ ]:
#print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())